In [1]:
import time
import json
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.firefox import GeckoDriverManager

BASE_URL = "https://projekty.ncn.gov.pl"
START_URL = "https://projekty.ncn.gov.pl/index.php?jednostka=Uniwersytet+Adama+Mickiewicza&jednostka_miasto=&jednostka_wojewodztwo=&kierownik=&kierownik_plec=&kierownik_tytul=&status=&projekt=&kwotaprzyznanaod=8+375&kwotaprzyznanado=7+209+600&typkonkursu=&konkurs=&grupa=&panel=&slowokluczowe=&aparatura="

options = webdriver.FirefoxOptions()
options.add_argument("--headless")  # jeśli chcesz widzieć przeglądarkę – zakomentuj to

driver = webdriver.Firefox(
    service=Service(GeckoDriverManager().install()),
    options=options
)


In [3]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver.get(START_URL)

wait = WebDriverWait(driver, 20)
# Zaczekaj aż na stronie pojawi się tekst "Wyniki wyszukiwania"
wait.until(EC.text_to_be_present_in_element((By.TAG_NAME, "body"), "Wyniki wyszukiwania"))

# Zbierz linki do szczegółów projektów
anchors = wait.until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href*='projekt_id=']"))
)

# Usuń duplikaty, zachowaj kolejność i weź pierwszą 10
links = []
for a in anchors:
    href = a.get_attribute("href")
    if href and "projekt_id=" in href and href not in links:
        links.append(href)
    if len(links) == 10:
        break

print("Znalezione linki:", len(links))
for i, href in enumerate(links, 1):
    print(f"{i}. {href}")


Znalezione linki: 10
1. https://projekty.ncn.gov.pl/index.php?projekt_id=337874
2. https://projekty.ncn.gov.pl/index.php?projekt_id=338215
3. https://projekty.ncn.gov.pl/index.php?projekt_id=342609
4. https://projekty.ncn.gov.pl/index.php?projekt_id=343528
5. https://projekty.ncn.gov.pl/index.php?projekt_id=344441
6. https://projekty.ncn.gov.pl/index.php?projekt_id=350342
7. https://projekty.ncn.gov.pl/index.php?projekt_id=351125
8. https://projekty.ncn.gov.pl/index.php?projekt_id=351555
9. https://projekty.ncn.gov.pl/index.php?projekt_id=351769
10. https://projekty.ncn.gov.pl/index.php?projekt_id=352730


In [13]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def parse_project(driver, url):
    driver.get(url)
    wait = WebDriverWait(driver, 15)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".important h2")))

    data = {}

    # Tytuł
    try:
        data["tytul"] = driver.find_element(By.CSS_SELECTOR, ".important h2").text
    except:
        data["tytul"] = None

    # ID projektu
    try:
        data["id"] = driver.find_element(By.CSS_SELECTOR, ".important p.row2").text
    except:
        data["id"] = None

    # Słowa kluczowe
    data["slowa_kluczowe"] = [el.text for el in driver.find_elements(By.CSS_SELECTOR, "p.row2 span.frazy a")]

    # Deskryptory
    data["deskryptory"] = [el.text for el in driver.find_elements(By.CSS_SELECTOR, ".important ul li")]

    # Panel
    try:
        data["panel"] = driver.find_element(By.CSS_SELECTOR, "p.wciecie").text
    except:
        data["panel"] = None

    # Jednostka realizująca
    try:
        data["jednostka"] = driver.find_element(
            By.XPATH, "//p[strong[contains(text(),'Jednostka realizująca')]]/following-sibling::p"
        ).text
    except:
        data["jednostka"] = None

    # Kierownik
    try:
        data["kierownik"] = driver.find_element(
            By.XPATH, "//p[strong[contains(text(),'Kierownik projektu')]]/following-sibling::p"
        ).text
    except:
        data["kierownik"] = None

    # Kwota, daty, status
    for el in driver.find_elements(By.CSS_SELECTOR, ".strona p"):
        txt = el.text
        if "Przyznana kwota" in txt: data["kwota"] = txt
        if "Rozpoczęcie projektu" in txt: data["start"] = txt
        if "Zakończenie projektu" in txt: data["koniec"] = txt
        if "Status projektu" in txt: data["status"] = txt

    # PDF opis
    # PDF opis
    try:
        pdf_url = driver.find_element(By.CSS_SELECTOR, "a[href$='.pdf']").get_attribute("href")
    except:
        pdf_url = None

    data["opis_pdf"] = pdf_url
    data["streszczenie"] = extract_pdf_text(pdf_url) if pdf_url else None


    # Publikacje
    publikacje = []
    for pub in driver.find_elements(By.CSS_SELECTOR, "li.publikacje"):
        pub_data = {}
        try: pub_data["tytul"] = pub.find_element(By.CSS_SELECTOR, ".tytul strong").text
        except: pub_data["tytul"] = None
        try: pub_data["autorzy"] = pub.find_element(By.CSS_SELECTOR, ".autorzy em").text
        except: pub_data["autorzy"] = None
        try: pub_data["czasopismo"] = pub.find_element(By.CSS_SELECTOR, ".czasopismo em").text
        except: pub_data["czasopismo"] = None
        try: pub_data["doi"] = pub.find_element(By.CSS_SELECTOR, ".doi .prawa").text
        except: pub_data["doi"] = None
        publikacje.append(pub_data)
        
    publikacje = [
        p for p in publikacje 
        if any([p["tytul"], p["autorzy"], p["czasopismo"], p["doi"]])
    ]
    data["publikacje"] = publikacje

    return data


In [14]:
import requests
import io
from PyPDF2 import PdfReader

def extract_pdf_text(pdf_url):
    try:
        headers = {"User-Agent": "Mozilla/5.0", "Referer": "https://projekty.ncn.gov.pl"}
        r = requests.get(pdf_url, headers=headers)
        r.raise_for_status()
        pdf_file = io.BytesIO(r.content)
        reader = PdfReader(pdf_file)

        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"

        return text.strip()
    except Exception as e:
        print("Błąd przy pobieraniu PDF:", e)
        return None


In [15]:
results = []
for url in links:
    print("Scrapuję:", url)
    dane = parse_project(driver, url)
    results.append(dane)

# Zapiszmy do JSON
import json
with open("projekty.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)


Scrapuję: https://projekty.ncn.gov.pl/index.php?projekt_id=337874
Scrapuję: https://projekty.ncn.gov.pl/index.php?projekt_id=338215
Scrapuję: https://projekty.ncn.gov.pl/index.php?projekt_id=342609
Scrapuję: https://projekty.ncn.gov.pl/index.php?projekt_id=343528
Scrapuję: https://projekty.ncn.gov.pl/index.php?projekt_id=344441
Scrapuję: https://projekty.ncn.gov.pl/index.php?projekt_id=350342
Scrapuję: https://projekty.ncn.gov.pl/index.php?projekt_id=351125
Scrapuję: https://projekty.ncn.gov.pl/index.php?projekt_id=351555
Scrapuję: https://projekty.ncn.gov.pl/index.php?projekt_id=351769
Scrapuję: https://projekty.ncn.gov.pl/index.php?projekt_id=352730
